In [1]:
import numpy as np              # NumPy, for working with arrays/tensors 
import time                     # For measuring time
import random                   # Python's random library
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# PyTorch libraries:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

if torch.cuda.is_available():
  print("Cuda (GPU support) is available and enabled!")
  device = torch.device("cuda")
else:
  print("Cuda (GPU support) is not available :(")
  device = torch.device("cpu")


Cuda (GPU support) is not available :(


In [2]:
LABELS_FILEPATH = "./SMAP MSL/labeled_anomalies.csv"
TRAINSET_FILEPATH = "./SMAP MSL/data/data/train"
TESTSET_FILEPATH = "./SMAP MSL/data/data/test"

In [3]:
labels = pd.read_csv(LABELS_FILEPATH)

In [4]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   chan_id            82 non-null     object
 1   spacecraft         82 non-null     object
 2   anomaly_sequences  82 non-null     object
 3   class              82 non-null     object
 4   num_values         82 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 3.3+ KB


In [5]:
labels.head(50)

,chan_id,spacecraft,anomaly_sequences,class,num_values
0,P-1,SMAP,"[[2149, 2349], [4536, 4844], [3539, 3779]]","[contextual, contextual, contextual]",8505
1,S-1,SMAP,"[[5300, 5747]]",[point],7331
2,E-1,SMAP,"[[5000, 5030], [5610, 6086]]","[contextual, contextual]",8516
3,E-2,SMAP,"[[5598, 6995]]",[point],8532
4,E-3,SMAP,"[[5094, 8306]]",[point],8307
5,E-4,SMAP,"[[5450, 8261]]",[point],8354
6,E-5,SMAP,"[[5600, 5920]]",[point],8294
7,E-6,SMAP,"[[5610, 5675]]",[point],8300
8,E-7,SMAP,"[[5394, 5674]]",[point],8310
9,E-8,SMAP,"[[5400, 6022]]",[point],8532


In [6]:
# Data loading 
# T-10 data is not in Labels but in Train Dataset. Therefore, T-10 will not be used for now.
import os
train_datas = {}
test_datas = {}
arranged_train_datas = {}
arranged_test_datas = {}

desiredSC      = 'SMAP'
# Iterate directory
for path in os.listdir(TRAINSET_FILEPATH):
    # check if current path is a file
    if os.path.isfile(os.path.join(TRAINSET_FILEPATH, path)):
        if path != 'T-10.npy':
            SC_labels     = labels[labels['chan_id'] == path.replace('.npy','')]
            SC            = SC_labels['spacecraft']
            # print("SC_labels",SC_labels)
            # print("SC",SC)
            # print((SC.iloc[0]),path.replace('.npy',''))
            SC_name       = SC.iloc[0]
            if SC_name == desiredSC:
                train_data = np.load(os.path.join(TRAINSET_FILEPATH, path))
                train_datas[path] = train_data

for path in os.listdir(TESTSET_FILEPATH):
    # check if current path is a file
    if os.path.isfile(os.path.join(TESTSET_FILEPATH, path)):
        if path != 'T-10.npy':
            SC_labels     = labels[labels['chan_id'] == path.replace('.npy','')]
            SC            = SC_labels['spacecraft']
            SC_name       = SC.iloc[0]
            if SC_name == desiredSC:       
                test_data = np.load(os.path.join(TESTSET_FILEPATH, path))
                test_datas[path] = test_data

In [7]:
# Training params
BATCH_SIZE = 128
INTERMEDIATE_LENGTH = 24
ALPHA  =0.1
DIMENSIONS  = 25 
lr = 0.005
#optimizer = torch.optim.Adam(lr = lr )
loss = nn.MSELoss()

In [8]:
## Writing Functions

def windowed_Set(original_data, window_size, shifting,horizon):
    time_sequence_length, n_dimensions  = original_data.shape
    T = int((time_sequence_length - window_size-horizon) / shifting + 1)
    
    windowedArray = np.empty((n_dimensions, T, window_size))
    futuredArray  = np.empty((n_dimensions, T, horizon))

    for d in range(n_dimensions):
        for i in range(T):
            start_index = i * shifting
            window       = original_data[start_index : start_index + window_size,d]
            horizon_data = original_data[start_index + window_size : start_index + window_size + horizon,d]
            windowedArray[d, i, :] = window
            futuredArray[d, i, :]  = horizon_data
    return windowedArray, futuredArray

def normalize_data(data_, desired_mean, desired_std):
    """This code turns the input into have the desired mean and variance values. When standard deviation is zero, 
       since division will be infinity, it will bypass.
    """
    # Calculate the mean and variance of the original data
    data = data_.copy()
    original_mean = np.mean(data)
    original_std = np.std(data)
     
    # Subtract the mean from each data point
    data -= original_mean
    # Divide each data point by the square root of the variance
    if original_std != 0:
        data /= original_std
        # Multiply each data point by the desired standard deviation
        data *= desired_std
    # Add the desired mean to each data point
    data += desired_mean
    return data


In [9]:
## Normalization Parameters
INPUT_LENGTH = 100
wL = INPUT_LENGTH     # window length
zeta = 1             # horizon length (future length to predict)
shift = 20

#4.1 data normalization 
#1 - training normalization 
train_means = {}
train_variances = {}
windowedTrain = {}      # Dividing the data with windows
futuredTrain = {}       # Creating arrays to be futured

for train_data in train_datas:
    #print(train_datas[train_data].shape)
    D                                       = train_datas[train_data].shape[1]
    iN                                      = train_datas[train_data].shape[0]

    for D_ in range(D):
        train_datas[train_data][:,D_]       = normalize_data(train_datas[train_data][:,D_],0,1)
    #train_datas[train_data] = train_datas[train_data] * 2 - 1  # variance 1 olacak teyit et
    
    windowedTrain[train_data], futuredTrain[train_data] = windowed_Set(train_datas[train_data],wL,shift,zeta)

print("Total set number is:",len(windowedTrain),
      "\n1 Set name:",train_data,
      "\nSet shape:",train_datas[train_data].shape,"[t,D]",
      "\nWindowed array shape:",windowedTrain[train_data].shape,"[D,i,w]",
      "\nFutured array shape:",futuredTrain[train_data].shape,"[D,i,zeta]",
      "\nOne sample mean:", np.mean(train_datas[train_data]),
      "\nOne sample dimension variance:", np.std(train_datas[train_data][:,0]))
      


Total set number is: 54 
1 Set name: T-3.npy 
Set shape: (2876, 25) [t,D] 
Windowed array shape: (25, 139, 100) [D,i,w] 
Futured array shape: (25, 139, 1) [D,i,zeta] 
One sample mean: 1.3242382001648598e-16 
One sample dimension variance: 1.0


In [10]:
windowedTest    = {}
futuredTest     = {}
test_means      = {}
test_variances  = {}
test_std        = {}

for test_data in test_datas:
    windowedTest[test_data], futuredTest[test_data] = windowed_Set(test_datas[test_data],wL,shift,zeta)

for test_data in test_datas:
    #print(test_datas[test_data].shape)
    D                               = windowedTest[test_data].shape[0]
    iN                              = windowedTest[test_data].shape[1]
    iN=2

    test_means[test_data]           = np.zeros((D,iN))
    test_variances[test_data]       = np.zeros((D,iN))
    test_std[test_data]             = np.zeros((D,iN))
    test_means[test_data][:,0]      = 0
    test_std[test_data][:,0]        = 0
    test_variances[test_data][:,0]  = 1 

    for iN_ in range(1,iN): 
        E                                = np.mean(windowedTest[test_data][:,iN_,:],axis = 1)
        E2                               = np.mean(windowedTest[test_data][:,iN_,:]**2,axis = 1)    
        test_means[test_data][:,iN_]     = (1-ALPHA)*test_means[test_data][:,iN_-1] + ALPHA*E 
        test_variances[test_data][:,iN_] = (1-ALPHA)*test_variances[test_data][:,iN_-1]+ALPHA*(E2-E**2)
        test_std[test_data][:,iN_]       = np.sqrt(test_variances[test_data][:,iN_-1])
        for D_ in range(D):
            dynamicMean                       = test_means[test_data][D_,iN_-1]
            dynamicVar                        = test_variances[test_data][D_,iN_-1]
            dynamicStd                        = test_std[test_data][D_,iN_-1]
            
            windowedTest[test_data][D_,iN_,:] = normalize_data(windowedTest[test_data][D_,iN_,:],dynamicMean,dynamicStd)

print("Window length:", wL,
      "\nShifting:",shift,
      "\nHorizon (zeta):",zeta,
      "\nTotal set number is:",len(windowedTest),
      "\n1 Set name:",test_data,
      "\nSet shape:",test_datas[test_data].shape,"[t,D]",
      "\nWindowed array shape:",windowedTest[test_data].shape,"[D,i,w]",
      "\nFutured array shape:",futuredTest[test_data].shape,"[D,i,zeta]",
      "\nTest means array shape:",test_means[test_data].shape,"[D,i]",
      "\nTest variance array shape:",test_variances[test_data].shape,"[D,i]")

Window length: 100 
Shifting: 20 
Horizon (zeta): 1 
Total set number is: 54 
1 Set name: T-3.npy 
Set shape: (8579, 25) [t,D] 
Windowed array shape: (25, 424, 100) [D,i,w] 
Futured array shape: (25, 424, 1) [D,i,zeta] 
Test means array shape: (25, 2) [D,i] 
Test variance array shape: (25, 2) [D,i]


In [11]:
print(train_datas.keys())
print('\n',test_datas.keys())
print('\n',labels['chan_id'].values)

dict_keys(['A-1.npy', 'A-2.npy', 'A-3.npy', 'A-4.npy', 'A-5.npy', 'A-6.npy', 'A-7.npy', 'A-8.npy', 'A-9.npy', 'B-1.npy', 'D-1.npy', 'D-11.npy', 'D-12.npy', 'D-13.npy', 'D-2.npy', 'D-3.npy', 'D-4.npy', 'D-5.npy', 'D-6.npy', 'D-7.npy', 'D-8.npy', 'D-9.npy', 'E-1.npy', 'E-10.npy', 'E-11.npy', 'E-12.npy', 'E-13.npy', 'E-2.npy', 'E-3.npy', 'E-4.npy', 'E-5.npy', 'E-6.npy', 'E-7.npy', 'E-8.npy', 'E-9.npy', 'F-1.npy', 'F-2.npy', 'F-3.npy', 'G-1.npy', 'G-2.npy', 'G-3.npy', 'G-4.npy', 'G-6.npy', 'G-7.npy', 'P-1.npy', 'P-2.npy', 'P-3.npy', 'P-4.npy', 'P-7.npy', 'R-1.npy', 'S-1.npy', 'T-1.npy', 'T-2.npy', 'T-3.npy'])

 dict_keys(['A-1.npy', 'A-2.npy', 'A-3.npy', 'A-4.npy', 'A-5.npy', 'A-6.npy', 'A-7.npy', 'A-8.npy', 'A-9.npy', 'B-1.npy', 'D-1.npy', 'D-11.npy', 'D-12.npy', 'D-13.npy', 'D-2.npy', 'D-3.npy', 'D-4.npy', 'D-5.npy', 'D-6.npy', 'D-7.npy', 'D-8.npy', 'D-9.npy', 'E-1.npy', 'E-10.npy', 'E-11.npy', 'E-12.npy', 'E-13.npy', 'E-2.npy', 'E-3.npy', 'E-4.npy', 'E-5.npy', 'E-6.npy', 'E-7.npy', 'E-8

In [17]:
INPUT_LENGTH

100

In [22]:
#4.2 temporal correlation
#inout sub block 
class InputSubBlock(nn.Module):

    def __init__(self):
        super(InputSubBlock, self).__init__()
        self.fc1 = nn.Linear(INPUT_LENGTH, 50)  # 5*5 from image dimension
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 50)
        self.fc4 = nn.Linear(50, 50)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
#cascade sub block 
class CascadeSubBlock(nn.Module):

    def __init__(self):
        super(CascadeSubBlock, self).__init__()
        self.fc1 = nn.Linear(50, 50)  # 5*5 from image dimension
        self.fc2 = nn.Linear(50, INPUT_LENGTH)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
#forecasting sub block
class ForecastingSubBlock(nn.Module):

    def __init__(self):
        super(ForecastingSubBlock, self).__init__()
        self.fc1 = nn.Linear(50, 50)  # 5*5 from image dimension
        self.fc2 = nn.Linear(50, INTERMEDIATE_LENGTH)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
#MLPBlock
class MLPBlock(nn.Module):

    def __init__(self):
        super(MLPBlock, self).__init__()
        self.input = InputSubBlock()
        self.cascade = CascadeSubBlock()
        self.forecasting = ForecastingSubBlock()

    def forward(self, x):
        #print("1",x.shape)
        x = self.input(x)
        #print("2s",x.shape)
        cascade = self.cascade(x)
        
        forecast = self.forecasting(x)
        
        return cascade , forecast
class TemporalModule(nn.Module):
    def __init__(self):
        super(TemporalModule, self).__init__()
        self.mlp1 = MLPBlock() 
        self.mlp2 = MLPBlock() 

    def forward(self, input):
        mlp1_out , forecast = self.mlp1(input)
        new_input = mlp1_out - input 
        mlp2_out , forecast_2 = self.mlp2(new_input)
        return (forecast + forecast_2)

s = TemporalModule()

In [13]:
from torch.utils.data import TensorDataset, DataLoader, Dataset
class MarinaDataset(Dataset):
    def __init__(self, vec,label):
        self.vec = vec
        self.label = label
    def __len__(self):
        return len(self.vec)
    def __getitem__(self, idx):        
        return self.vec[idx], self.label[idx] 

In [14]:
# Stacking all data
trainDataT      = []    # Create a list for training data
futuredTrainT   = []    
for i,name in enumerate(windowedTrain.keys()):
    for j in range(windowedTrain[name].shape[1]):                                 # Looping over T dimension for stacking                              
        trainDataT.append(torch.tensor(windowedTrain[name][:,0,:]).float())   # Swapping the dimensions accordingly as (T,D,W).
        futuredTrainT.append(torch.tensor(futuredTrain[name][:,0,:]).float()) # Swapping the dimensions accordingly as (T,D,W).

    #print(windowedTrain[name].shape)
print("List length of trainDataT:",len(trainDataT),", and shape of",trainDataT[1].shape,
      "\nLength of futured (targets) list:",len(futuredTrainT),", and shape of",futuredTrainT[1].shape)

List length of trainDataT: 6650 , and shape of torch.Size([25, 100]) 
Length of futured (targets) list: 6650 , and shape of torch.Size([25, 1])


In [15]:
trainset            = MarinaDataset(trainDataT,futuredTrainT)
trainloader         = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True)                                    

In [23]:
out1 = s(trainDataT[0])
print(out1.shape)

torch.Size([25, 24])
